In [2]:
from itertools import product
from collections import deque
import multiprocessing

from ipywidgets import IntProgress
from IPython.display import display

from src.pid import PIDController
from src.simulation import Simulation
from src.genetic import fitness

In [3]:
p = [x / 10.0 for x in range(-100, 101, 5)]
i = [x / 500000.0 for x in range(-100, 101, 10)]
d = [x / 0.2 for x in range(-100, 101, 5)]

In [4]:
pid_combinations = list(product(p, i, d))
num_pid_combinations = len(pid_combinations)

num_pid_combinations, pid_combinations[:2]

(35301, [(-10.0, -0.0002, -500.0), (-10.0, -0.0002, -475.0)])

In [5]:
setpoints = [-5] * 30 * 2 + [7] * 30 * 2

In [18]:
f = IntProgress(min=0, max=num_pid_combinations / 1000)
display(f)

best_parameter_error = 99999999
positions = deque()
times = deque()
simulation_times = deque()
for idx, (kp, ki, kd) in enumerate(pid_combinations):
    pid_controller = PIDController(kp, ki, kd, setpoints[0])
    simulation = Simulation(mass=0.2, delta_t=1 / 30)
    position = 0
    positions = []
    
    for setpoint in setpoints:
        pid_controller.setpoint = setpoint
        new_angle = pid_controller.next(position)
        _, _, position = simulation.next(new_angle)
        positions.append(position)

    error = fitness(positions, setpoints, weight_factor=2)
    if error < best_parameter_error:
        best_parameters = kp, ki, kd
        best_parameter_error = error
        
    if (idx + 1) % 1000 == 0:
        f.value += 1

IntProgress(value=0, max=35)

In [19]:
best_parameters

(-8.0, 0.0002, -125.0)

In [36]:
multiprocessing.cpu_count()

8